## Working with India's DIGIPIN in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/19_digipin.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/14_DIGIPIN.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/14_DIGIPIN.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/14_DIGIPIN.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/14_DIGIPIN.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### Latlon to DIGIPIN

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2digipin

lat = 17.414718
lon = 78.482992 
res = 10
digipin_id = latlon2digipin(lat, lon, res)
digipin_id

### DIGIPIN to Polygon

In [ ]:
from vgrid.conversion.dggs2geo.digipin2geo import digipin2geo

digipin_geo = digipin2geo(digipin_id)
digipin_geo

### DIGIPIN to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.digipin2geo import digipin2geojson

digipin_geojson = digipin2geojson(digipin_id)
digipin_geojson

### Vector to DIGIPIN

In [ ]:
from vgrid.conversion.vector2dggs.vector2digipin import vector2digipin

file_path = "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/india_polygon.geojson"
vector_to_digipin = vector2digipin(
    file_path,
    compact=False,
    resolution=5,
    topology=False,
    predicate="intersects",
    output_format="gpd",
)
# Visualize the output

vector_to_digipin.plot(edgecolor="white")

### DIGIPIN Compact

In [ ]:
from vgrid.conversion.dggscompact.digipincompact import digipincompact

digipin_compacted = digipincompact(
    vector_to_digipin, digipin_id="digipin", output_format="gpd"
)
digipin_compacted.plot(edgecolor="white")

### DIGIPIN Expand

In [ ]:
from vgrid.conversion.dggscompact.digipincompact import digipinexpand

digipin_expanded = digipinexpand(
    vector_to_digipin, resolution=6, output_format="gpd"
)
digipin_expanded.plot(edgecolor="white")

### DIGIPIN Binning

In [ ]:
from vgrid.binning.digipinbin import digipinbin

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/india_point.geojson"
)
stats = "count"
digipin_bin = digipinbin(
    file_path,
    resolution=5,
    stats=stats,
    output_format="gpd",
)
digipin_bin.plot(
    column=stats,  # numeric column to base the colors on
    cmap="Spectral_r",  # color scheme (matplotlib colormap)
    legend=True,
    linewidth=0.2,  # boundary width (optional)
)

### Raster to DIGIPIN

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/india_rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, "r")
print(src.meta)
show(src)

#### Convert raster to DIGIPIN

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2digipin import raster2digipin

raster_to_digipin = raster2digipin(raster_file, resolution = 8, output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=25)

digipin_layer = folium.GeoJson(
    raster_to_digipin,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["digipin", "band_1", "band_2", "band_3"],
        aliases=["DIGIPIN ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(digipin_layer.get_bounds())

# Display the map
m

### DIGIPIN Generator

In [ ]:
from vgrid.generator.digipingrid import digipingrid
bbox=[77.95,17.01,79.0, 17.98]
digipin_grid = digipingrid(resolution=5,bbox = bbox, output_format="gpd")
digipin_grid.plot(edgecolor="white")

### DIGIPIN Statistics

Characteristic Length Scale (CLS - suggested by Ralph Kahn): the diameter of a spherical cap of the same cell's area

In [ ]:
from vgrid.stats import digipinstats

digipinstats("m")